Copyright (c) Microsoft Corporation. Licensed under the MIT license.

### Lab 03b: Semantic Kernel Basic Concepts (NL2SQL Scenario)  

**Objective:** In this lab, we will develop a basic understanding of Semantic Kernel concepts by creating a Natural Language to SQL (NL2SQL) scenario. 

We will use Azure's OpenAI service to implement a pipeline that translates natural language queries into SQL commands to interact with a database.

#### Step 1: Set Up Azure OpenAI Service  
- **Create an Azure OpenAI service** with a GPT-4 or GPT-3.5-Turbo deployment using the Azure Portal. GPT-4 is recommended for optimal performance.  
- **Configure environment variables** for Semantic Kernel to connect to this service by creating an `.env` file. Use the provided `.env.template` as a starting point, rename it to `.env`, and replace the placeholder values with your actual service details.  

#### Step 2: Create the Database  
In this section, we will establish a database connection, define our database schema with tables, and populate the tables with data.  

In [1]:
from sqlalchemy import create_engine, Column, Integer, String, ForeignKey
from sqlalchemy.orm import sessionmaker, declarative_base
import pandas as pd

# Create the SQLite database
engine = create_engine('sqlite:///data.db')

# Create tables
Base = declarative_base()

class City(Base):
    __tablename__ = 'cities'
    city_id = Column(Integer, primary_key=True)
    name = Column(String)

class Product(Base):
    __tablename__ = 'products'
    product_id = Column(Integer, primary_key=True)
    name = Column(String)

class Sale(Base):
    __tablename__ = 'sales'
    sale_id = Column(Integer, primary_key=True)  # Add this line
    city_id = Column(Integer, ForeignKey('cities.city_id'))
    product_id = Column(Integer, ForeignKey('products.product_id'))
    total = Column(Integer)

Base.metadata.create_all(engine)

# Create a session
Session = sessionmaker(bind=engine)
session = Session()

# Clear tables
session.query(City).delete()
session.query(Product).delete()
session.query(Sale).delete()
session.commit()

# Load data
try:
    df_city = pd.read_csv('data/cities.csv')
    df_city.to_sql('cities', con=engine, if_exists='append', index=False)
except Exception as e:
    print(f"Error loading cities data: {e}")

try:
    df_product = pd.read_csv('data/products.csv')
    df_product.to_sql('products', con=engine, if_exists='append', index=False)
except Exception as e:
    print(f"Error loading products data: {e}")

try:
    df_sale = pd.read_csv('data/sales.csv')
    df_sale.to_sql('sales', con=engine, if_exists='append', index=False)
except Exception as e:
    print(f"Error loading sales data: {e}")


#### Step 3: Understanding the NL2SQL Pipeline  

The typical NL2SQL pipeline consists of the following steps:  

1. 💻 Obtain the database schema  
2. 🧠 Select relevant tables and columns based on the schema  
3. 🧠 Generate the SQL query  
4. 💻 Execute the SQL query against the database  
5. 🧠 Generate a human-readable answer from the query result  
   
We will implement an **NL2SQL plugin** that encapsulates the functions for each step above.  

#### Step 4: Create the NL2SQL Plugin Directory  

To organize our NL2SQL functions, we create a dedicated directory within our project.  

In [2]:
!mkdir -p plugins/NL2SQL

#### Step 5: Implementing NL2SQL Functions  
We will now create individual functions for each step in the NL2SQL pipeline.  
   
💻 Get Database Schema Function  

This function retrieves the schema of the database, including table names, columns, primary keys, and foreign keys.  

In [3]:
%%writefile plugins/NL2SQL/native_function.py
from semantic_kernel.skill_definition import sk_function
from semantic_kernel import SKContext
import json
from sqlalchemy import inspect, create_engine

class NL2SQL:
    @sk_function(
        description="Get the database schema",
        name="GetDatabaseSchema",
        input_description="The user question",
    )
    def get_database_schema(self, context: SKContext) -> str:
        engine = create_engine('sqlite:///data.db')
        
        # Create inspector
        inspector = inspect(engine)

        # Get table information
        tables = inspector.get_table_names()

        # Store table information
        table_info = {}

        for table in tables:
            table_info[table] = {
                "columns": [],
                "primary_key": [],
                "foreign_keys": []
            }
            
            # Get columns
            columns = inspector.get_columns(table)
            for column in columns:
                table_info[table]["columns"].append({
                    "name": column['name'],
                    "type": str(column['type'])
                })
            
            # Get primary key
            pk = inspector.get_pk_constraint(table)['constrained_columns']
            table_info[table]["primary_key"] = pk
            
            # Get foreign keys
            fks = inspector.get_foreign_keys(table)
            if fks:
                for fk in fks:
                    table_info[table]["foreign_keys"].append({
                        "columns": fk['constrained_columns'],
                        "referred_table": fk['referred_table'],
                        "referred_columns": fk['referred_columns']
                    })

        # Convert to JSON
        schema = json.dumps(table_info, indent=4)
        context["schema"] = schema

        return schema        

Overwriting plugins/NL2SQL/native_function.py


Test 💻 **Get database schema** function

In [3]:
import semantic_kernel as sk
from plugins.NL2SQL.native_function import NL2SQL
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion

kernel = sk.Kernel()
deployment, api_key, endpoint, api_version = sk.azure_openai_settings_from_dot_env(include_api_version=True)
kernel.add_chat_service("chat-completion", AzureChatCompletion(deployment_name=deployment, endpoint=endpoint, api_key=api_key, api_version=api_version))
 
# import the plugin
nl2sql_plugin = kernel.import_skill(NL2SQL(), skill_name="NL2SQL")

# Run the GetDatabaseSchema function with the context.
context = kernel.create_new_context()
output_context = await kernel.run_async(
    nl2sql_plugin["GetDatabaseSchema"], input_context=context
)

print(output_context["schema"])

{
    "cities": {
        "columns": [
            {
                "name": "city_id",
                "type": "INTEGER"
            },
            {
                "name": "name",
                "type": "VARCHAR"
            }
        ],
        "primary_key": [
            "city_id"
        ],
        "foreign_keys": []
    },
    "products": {
        "columns": [
            {
                "name": "product_id",
                "type": "INTEGER"
            },
            {
                "name": "name",
                "type": "VARCHAR"
            }
        ],
        "primary_key": [
            "product_id"
        ],
        "foreign_keys": []
    },
    "sales": {
        "columns": [
            {
                "name": "sale_id",
                "type": "INTEGER"
            },
            {
                "name": "city_id",
                "type": "INTEGER"
            },
            {
                "name": "product_id",
                "type": "INTEGER"
        

#### 🧠 Select Tables and Columns Function  

Based on the user's natural language query and the database schema, this function identifies the relevant schema elements to construct an SQL query.

First create the directory for the function

In [4]:

!mkdir -p plugins/NL2SQL/SelectTablesAndColumns

Create function config file

In [5]:
%%writefile plugins/NL2SQL/SelectTablesAndColumns/config.json
{
     "schema": 1,
     "type": "completion",
     "description": "Based on the user ask and the database schema select the database schema elements that are related to the user ask.",
     "completion": {
          "max_tokens": 500,
          "temperature": 0.0,
          "top_p": 0.0,
          "presence_penalty": 0.0,
          "frequency_penalty": 0.0
     },
     "input": {
          "parameters": [
               {
                    "name": "ask",
                    "description": "The user's ask.",
                    "defaultValue": "",
                    "required": true
               },
               {
                    "name": "schema",
                    "description": "The database schema obtained with GetDatabaseSchema.",
                    "defaultValue": "",
                    "required": true
               }               
          ]
     }
}

Overwriting plugins/NL2SQL/SelectTablesAndColumns/config.json


Create function prompt

In [6]:
%%writefile plugins/NL2SQL/SelectTablesAndColumns/skprompt.txt
## Task Goal:
Based on a user's ask and a database schema elements (tables, columns, primary and foreign keys) identify which elements from the database schema are linked to the user ask so that, subsequently, they can be used int the generation of a SQL query.

## Task Instructions:
Output format is a list only do not write output word to the output.

## Examples: 
user ask: "I want to know the name of all subscribers of plans whose data allowance is less than 100 minutes."
schema: {"subscribers": {"columns": [{"name": "subscriber_id", "type": "INTEGER"}, {"name": "name", "type": "VARCHAR"}], "primary_key": ["subscriber_id"], "foreign_keys": []}, "plans": {"columns": [{"name": "plan_id", "type": "INTEGER"}, {"name": "name", "type": "VARCHAR"}, {"name": "data_allowance", "type": "INTEGER"}], "primary_key": ["plan_id"], "foreign_keys": []}, "subscribers_plans": {"columns": [{"name": "subscription_id", "type": "INTEGER"}, {"name": "plan_id", "type": "INTEGER"}, {"name": "subscriber_id", "type": "INTEGER"}], "primary_key": ["subscription_id"], "foreign_keys": [{"columns": ["plan_id"], "referred_table": "plans", "referred_columns": ["plan_id"]}, {"columns": ["subscriber_id"], "referred_table": "subscribers", "referred_columns": ["subscriber_id"]}]}}
output: ["subscribers.name", "plans.data_allowance", 100]

user ask: "List the number of subscribers by plan."
schema: {"subscribers": {"columns": [{"name": "subscriber_id", "type": "INTEGER"}, {"name": "name", "type": "VARCHAR"}], "primary_key": ["subscriber_id"], "foreign_keys": []}, "plans": {"columns": [{"name": "plan_id", "type": "INTEGER"}, {"name": "name", "type": "VARCHAR"}, {"name": "data_allowance", "type": "INTEGER"}], "primary_key": ["plan_id"], "foreign_keys": []}, "subscribers_plans": {"columns": [{"name": "subscription_id", "type": "INTEGER"}, {"name": "plan_id", "type": "INTEGER"}, {"name": "subscriber_id", "type": "INTEGER"}], "primary_key": ["subscription_id"], "foreign_keys": [{"columns": ["plan_id"], "referred_table": "plans", "referred_columns": ["plan_id"]}, {"columns": ["subscriber_id"], "referred_table": "subscribers", "referred_columns": ["subscriber_id"]}]}}
output: ["subscribers.subscriber_id", "plans.name"]

## User Question
user ask: "{{$ask}}"
schema: {{$schema}}
output: 

Overwriting plugins/NL2SQL/SelectTablesAndColumns/skprompt.txt


Test 🧠 **Select tables and columns** function

In [7]:
import semantic_kernel as sk

# Initialize the kernel
kernel = sk.Kernel()
deployment, api_key, endpoint, api_version = sk.azure_openai_settings_from_dot_env(include_api_version=True)
kernel.add_chat_service("chat-completion", AzureChatCompletion(deployment_name=deployment, endpoint=endpoint, api_key=api_key, api_version=api_version))

# Import the NL2SQL from the plugins directory.
nl2sql_plugin = kernel.import_semantic_skill_from_directory("./plugins", "NL2SQL")

context = kernel.create_new_context()
context.variables["ask"] = "Total sales in Houton?"
context.variables["schema"] = '{"cities": {"columns": [{"name": "city_id", "type": "INTEGER"}, {"name": "name", "type": "VARCHAR"}], "primary_key": ["city_id"], "foreign_keys": []}, "products": {"columns": [{"name": "product_id", "type": "INTEGER"}, {"name": "name", "type": "VARCHAR"}], "primary_key": ["product_id"], "foreign_keys": []}, "sales": {"columns": [{"name": "sale_id", "type": "INTEGER"}, {"name": "city_id", "type": "INTEGER"}, {"name": "product_id", "type": "INTEGER"}, {"name": "total", "type": "INTEGER"}], "primary_key": ["sale_id"], "foreign_keys": [{"columns": ["city_id"], "referred_table": "cities", "referred_columns": ["city_id"]}, {"columns": ["product_id"], "referred_table": "products", "referred_columns": ["product_id"]}]}}'

# Run the SelectTablesAndColumns function.
output_context = await kernel.run_async(
    nl2sql_plugin["SelectTablesAndColumns"],
    input_context = context
)
schema_links = output_context.result
print(schema_links)

["sales.total", "cities.name", "Houston"]


#### 🧠 Create SQL Query Function  

Given the selected schema elements and the user's query, this function generates the appropriate SQL command.

First create the directory for the function

In [8]:
!mkdir -p plugins/NL2SQL/GenerateQuery

Create function config file

In [10]:
%%writefile plugins/NL2SQL/GenerateQuery/config.json
{
     "schema": 1,
     "type": "completion",
     "description": "Based on the user ask, the database schema and the selected database schema elements generate the sql query that will be executed to meet the user ask.",
     "completion": {
          "max_tokens": 500,
          "temperature": 0.0,
          "top_p": 0.0,
          "presence_penalty": 0.0,
          "frequency_penalty": 0.0
     },
     "input": {
          "parameters": [
               {
                    "name": "ask",
                    "description": "The user's ask.",
                    "defaultValue": "",
                    "required": true
               },
               {
                    "name": "schema",
                    "description": "The database schema.",
                    "defaultValue": "",
                    "required": true
               },
               {
                    "name": "input",
                    "description": "The selected elements in the schema that are linked to the ask.",
                    "defaultValue": "",
                    "required": true
               }                     
          ]
     }
}

Overwriting plugins/NL2SQL/GenerateQuery/config.json


Create function prompt


In [11]:
%%writefile plugins/NL2SQL/GenerateQuery/skprompt.txt

## Task Goal
Given the user's ask, use the tables, columns, primary keys, and foreign keys, and the schema_links to generate a database query that can be executed in a SQLite database. 

## Task Instructions
Follow the following rules in the construction of the SQL:  
1) Use the database values that are explicitly mentioned in the question.  
2) Pay attention to the columns used to make the JOIN between tables to always use the Foreign Keys (FK).  
3) Use DESC and DISTINCT when necessary.  
4) Pay attention to the columns used in the GROUP BY command.  
5) Pay attention to the columns used in the SELECT command.  
6) Only change the GROUP BY command if necessary (avoid redundant columns in GROUP BY)  
Output is a valid SQL query for SQLite. Do not add explanations of the steps neither the word sql to the output.

## Examples: 
user ask: "I want to know the name of all subscribers of plans whose data allowance is less than 100 minutes."
schema_links: ["subscribers.name", "plan.data_allowance", 100]
schema: {"subscribers": {"columns": [{"name": "subscriber_id", "type": "INTEGER"}, {"name": "name", "type": "VARCHAR"}], "primary_key": ["subscriber_id"], "foreign_keys": []}, "plans": {"columns": [{"name": "plan_id", "type": "INTEGER"}, {"name": "name", "type": "VARCHAR"}, {"name": "data_allowance", "type": "INTEGER"}], "primary_key": ["plan_id"], "foreign_keys": []}, "subscribers_plans": {"columns": [{"name": "subscription_id", "type": "INTEGER"}, {"name": "plan_id", "type": "INTEGER"}, {"name": "subscriber_id", "type": "INTEGER"}], "primary_key": ["subscription_id"], "foreign_keys": [{"columns": ["plan_id"], "referred_table": "plans", "referred_columns": ["plan_id"]}, {"columns": ["subscriber_id"], "referred_table": "subscribers", "referred_columns": ["subscriber_id"]}]}}
output: SELECT Assinante.* FROM Assinante JOIN Assinatura ON Assinante.id = Assinatura.assinante_id JOIN Plano ON Assinatura.plano_id = Plano.id WHERE Plano.franquia_dados < 100

## User question: 
user ask: {{$ask}}
schema_links: {{$input}}
schema: {{$schema}}
output:

Overwriting plugins/NL2SQL/GenerateQuery/skprompt.txt


Test 🧠 **Generate query** function

Executes the generated SQL query against the database and retrieves the results.  

In [12]:
import semantic_kernel as sk

# Initialize the kernel
kernel = sk.Kernel()
deployment, api_key, endpoint, api_version = sk.azure_openai_settings_from_dot_env(include_api_version=True)
kernel.add_chat_service("chat-completion", AzureChatCompletion(deployment_name=deployment, endpoint=endpoint, api_key=api_key, api_version=api_version))

# Import the NL2SQL from the plugins directory.
nl2sql_plugin = kernel.import_semantic_skill_from_directory("./plugins", "NL2SQL")

context = kernel.create_new_context()
context.variables["ask"] = "Total sales in Houston?"
context.variables["schema"] = '{"cities": {"columns": [{"name": "city_id", "type": "INTEGER"}, {"name": "name", "type": "VARCHAR"}], "primary_key": ["city_id"], "foreign_keys": []}, "products": {"columns": [{"name": "product_id", "type": "INTEGER"}, {"name": "name", "type": "VARCHAR"}], "primary_key": ["product_id"], "foreign_keys": []}, "sales": {"columns": [{"name": "sale_id", "type": "INTEGER"}, {"name": "city_id", "type": "INTEGER"}, {"name": "product_id", "type": "INTEGER"}, {"name": "total", "type": "INTEGER"}], "primary_key": ["sale_id"], "foreign_keys": [{"columns": ["city_id"], "referred_table": "cities", "referred_columns": ["city_id"]}, {"columns": ["product_id"], "referred_table": "products", "referred_columns": ["product_id"]}]}}'
context.variables["input"] = '["sales.total", "cities.name", "Houston"]'

# Run the GenerateQuery function.
output_context = await kernel.run_async(
    nl2sql_plugin["GenerateQuery"],
    input_context = context
)

print(output_context.result)

SELECT SUM(sales.total) FROM sales JOIN cities ON sales.city_id = cities.city_id WHERE cities.name = 'Houston'


Create 💻 **Execute query** function

In [15]:
%%writefile plugins/NL2SQL/native_function.py
from semantic_kernel.skill_definition import sk_function
from semantic_kernel import SKContext
import json
from sqlalchemy import inspect, create_engine, text

class NL2SQL:
    @sk_function(
        description="Execute the generated sql query to have its results used in the answer generation",
        name="ExecuteQuery",
        input_description="The sql instruction",
    )
    def execute_query(self, context: SKContext) -> str:
        engine = create_engine('sqlite:///data.db')
        connection = engine.connect()
        query_results = ""
        sql_query = context["input"]
        context["sql_query"] = sql_query # add the sql query to the context        
        try:
            sql_result = connection.execute(text(sql_query))
            rows = sql_result.fetchall()  # Fetch all rows
            for row in rows:
                query_results = query_results + ', '.join(map(str, row)) + '\n' 
        except Exception as e:
            print(f"Error executing query: {e}")
        finally:
            connection.close()
        return query_results

    @sk_function(
        description="Get the database schema",
        name="GetDatabaseSchema",
        input_description="The user question",
    )
    def get_database_schema(self, context: SKContext) -> str:
        engine = create_engine('sqlite:///data.db')
        
        # Create inspector
        inspector = inspect(engine)

        # Get table information
        tables = inspector.get_table_names()

        # Store table information
        table_info = {}

        for table in tables:
            table_info[table] = {
                "columns": [],
                "primary_key": [],
                "foreign_keys": []
            }
            
            # Get columns
            columns = inspector.get_columns(table)
            for column in columns:
                table_info[table]["columns"].append({
                    "name": column['name'],
                    "type": str(column['type'])
                })
            
            # Get primary key
            pk = inspector.get_pk_constraint(table)['constrained_columns']
            table_info[table]["primary_key"] = pk
            
            # Get foreign keys
            fks = inspector.get_foreign_keys(table)
            if fks:
                for fk in fks:
                    table_info[table]["foreign_keys"].append({
                        "columns": fk['constrained_columns'],
                        "referred_table": fk['referred_table'],
                        "referred_columns": fk['referred_columns']
                    })

        # Convert to JSON
        schema = json.dumps(table_info, indent=4)
        context["schema"] = schema

        return schema      

Overwriting plugins/NL2SQL/native_function.py


Test 💻 **Execute query** function

In [16]:
import importlib
import semantic_kernel as sk
import plugins.NL2SQL.native_function as nl2sql
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion

# Reload the NL2SQL module
importlib.reload(nl2sql)

kernel = sk.Kernel()
deployment, api_key, endpoint, api_version = sk.azure_openai_settings_from_dot_env(include_api_version=True)
kernel.add_chat_service("chat-completion", AzureChatCompletion(deployment_name=deployment, endpoint=endpoint, api_key=api_key, api_version=api_version))
 
# import the plugin
nl2sql_plugin = kernel.import_skill(nl2sql.NL2SQL(), skill_name="NL2SQL")

contex = kernel.create_new_context()
context.variables["input"] = "SELECT SUM(sales.total) FROM sales JOIN cities ON sales.city_id = cities.city_id WHERE cities.name = 'Houston'"

# Run the ExecuteQuery function.
output_context = await kernel.run_async(
    nl2sql_plugin["ExecuteQuery"],
    input_context = context
)
output_context.result
print(output_context.result)

1520



#### 🧠 Generate Answer Function  

Finally, this function formats the query results into a human-readable answer to the user's original query.  

Create function directory


In [17]:
!mkdir -p plugins/NL2SQL/GenerateAnswer

Create function config file

In [19]:
%%writefile plugins/NL2SQL/GenerateAnswer/config.json
{
     "schema": 1,
     "type": "completion",
     "description": "Based on the user ask, the query and the query results generate an answer to the user ask.",
     "completion": {
          "max_tokens": 500,
          "temperature": 0.0,
          "top_p": 0.0,
          "presence_penalty": 0.0,
          "frequency_penalty": 0.0
     },
     "input": {
          "parameters": [
               {
                    "name": "ask",
                    "description": "The user's ask.",
                    "defaultValue": "",
                    "required": true
               },
               {
                    "name": "sql_query",
                    "description": "The sql query.",
                    "defaultValue": "",
                    "required": true
               },
               {
                    "name": "input",
                    "description": "The sql query result.",
                    "defaultValue": "",
                    "required": true
               }                     
          ]
     }
}

Overwriting plugins/NL2SQL/GenerateAnswer/config.json


In [21]:
%%writefile plugins/NL2SQL/GenerateAnswer/skprompt.txt

## Task Goal
Given the user's ask, the sql query and the query results generate an answer to the user. 

## User question: 
user ask: {{$ask}}
sql_query: {{$sql_query}}
sql_query_results: {{$input}}
answer:

Overwriting plugins/NL2SQL/GenerateAnswer/skprompt.txt


Test 🧠 **Generate answer** function

In [22]:
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion

kernel = sk.Kernel()
deployment, api_key, endpoint, api_version = sk.azure_openai_settings_from_dot_env(include_api_version=True)
kernel.add_chat_service("chat-completion", AzureChatCompletion(deployment_name=deployment, endpoint=endpoint, api_key=api_key, api_version=api_version))
 
# import the plugin
nl2sql_plugin = kernel.import_semantic_skill_from_directory("./plugins", "NL2SQL")

context = kernel.create_new_context()
context.variables["ask"] = "Total sales in Houston?"
context.variables["sql_query"] = "SELECT SUM(sales.total) FROM sales JOIN cities ON sales.city_id = cities.city_id WHERE cities.name = 'Houston'"
context.variables["input"] = "1520"

# Run the GenerateAnswer function.
output_context = await kernel.run_async(
    nl2sql_plugin["GenerateAnswer"],
    input_context = context
)
output_context.result
print(output_context.result)

The total sales in Houston are $1,520.


## Step 6: Chaining Functions in a Pipeline  

We will chain the functions together to form a complete NL2SQL pipeline that can process a natural language query and return an answer.  


In [24]:
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion

kernel = sk.Kernel()
deployment, api_key, endpoint, api_version = sk.azure_openai_settings_from_dot_env(include_api_version=True)
kernel.add_chat_service("chat-completion", AzureChatCompletion(deployment_name=deployment, endpoint=endpoint, api_key=api_key, api_version=api_version))

# import the plugin semantic and native functions
nl2sql_plugin = kernel.import_semantic_skill_from_directory("./plugins", "NL2SQL")
native_functions = kernel.import_native_skill_from_directory("./plugins", "NL2SQL")
nl2sql_plugin.update(native_functions)

# create new context
context = kernel.create_new_context()
# context.variables["ask"] = "What are the names of the cities?"
context.variables["ask"] = "total sales in Houston?"

output_context = await kernel.run_async(nl2sql_plugin["GetDatabaseSchema"], nl2sql_plugin["SelectTablesAndColumns"], nl2sql_plugin["GenerateQuery"], nl2sql_plugin["ExecuteQuery"], nl2sql_plugin["GenerateAnswer"], input_context=context)
print(output_context.result)

The total sales in Houston are $1,520.


## Step 7: Using a Planner  

A planner can be used to automatically generate and execute a plan based on the user's query. It will orchestrate the NL2SQL functions to achieve

In [25]:
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.planning import SequentialPlanner

kernel = sk.Kernel()
deployment, api_key, endpoint, api_version = sk.azure_openai_settings_from_dot_env(include_api_version=True)
kernel.add_chat_service("chat-completion", AzureChatCompletion(deployment_name=deployment, endpoint=endpoint, api_key=api_key, api_version=api_version))

# import the plugin semantic and native functions
kernel.import_semantic_skill_from_directory("./plugins", "NL2SQL")
kernel.import_native_skill_from_directory("./plugins", "NL2SQL")

planner = SequentialPlanner(kernel)

ask = "Total sales in Houston?"
# ask = "What are the names of the cities?"

# Create the plan
plan = await planner.create_plan_async(goal=ask)
print("Generated Plan:")
for step in plan._steps:
    print(step.skill_name, step.name, ":", step.description)

# Execute the plan
try:
    result = await plan.invoke_async()
    print("\nPlan results:")
    print(result)
except Exception as e:
    print(f"Error executing plan: {e}")

Generated Plan:
NL2SQL GetDatabaseSchema : Get the database schema
NL2SQL SelectTablesAndColumns : Based on the user ask and the database schema select the database schema elements that are related to the user ask.
NL2SQL GenerateQuery : Based on the user ask, the database schema and the selected database schema elements generate the sql query that will be executed to meet the user ask.
NL2SQL ExecuteQuery : Execute the generated sql query to have its results used in the answer generation
NL2SQL GenerateAnswer : Based on the user ask, the query and the query results generate an answer to the user ask.

Plan results:
The total sales in Houston are $1,520.
